# Jacobo Casado de Gracia 
Apartado de entrenamiento y evaluación de los modelos del trabajo de clasificación de Diatomeas. Implementado en Google Colab.

# Importación de librerías necesarias para el proyecto.

In [ ]:
import numpy as np
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
import shutil
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax, SGD
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from keras.applications.efficientnet import *
from sklearn.model_selection import StratifiedKFold

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

En caso de tener los datos en un comprimido .zip, se deja el comando:

In [ ]:
#!unzip -q "/content/drive/MyDrive/data_big.zip" -d "/content/drive/MyDrive/data_big/"

Funciones ya comentadas en el otro script llamado train_test_val_creation y que se adjuntan y utilizan aquí. \\
Se podría utilizar un dataframe de un fichero pero se cargan los datos de nuevo debido a que el proceso de entrenamiento es más rápido así. \\
Si se desea más información acerca de las funciones, recomiendo consultar el fichero comentado anteriormente donde se explica lo que realiza cada una de ellas.

In [ ]:
def get_dataframe(sdir, dir):

    ht  = 0
    wt = 0
    samples = 0
    sample_count = 10
    filepaths = []
    labels = []
    classlist = os.listdir(sdir)


    for klass in classlist:

        classpath = os.path.join(sdir, klass)
        flist = os.listdir(classpath)

        for i, f in enumerate(flist):

            fpath=os.path.join(classpath,f)

            try: 
                img = plt.imread(fpath)
                shape = img.shape
                filepaths.append(fpath)
                labels.append(klass)  


                if i < sample_count:
                    img = plt.imread(fpath)               
                    ht += img.shape[0]
                    wt += img.shape[1]
                    samples += 1
            except:
                print ('El archivo ', fpath, ' es una imagen inválida. ')

    filepaths = pd.Series(filepaths, name='filepaths')
    labels = pd.Series(labels, name='labels')

    df = pd.concat([filepaths, labels], axis = 1)
    class_count = len(df['labels'].unique())
    print('El df tiene ', class_count, ' clases.')

    groups = df.groupby('labels')
    print('{0:^30s} {1:^13s}'.format('Clase', 'Cantidad de imágenes'))

    

    for label in df['labels'].unique():
          group = groups.get_group(label)      
          print('{0:^30s} {1:^13s}'.format(label, str(len(group))))

    wave = wt/samples
    have = ht/samples
    aspect_ratio = have/wave
    print ('Altura media: ' ,have, '  Anchura media: ', wave, '  Aspect ratio medio: ', aspect_ratio)
    
    return df

def trim (df, max_size, min_size):
    
    column = 'labels'
    df = df.copy()
    original_class_count = len(list(df[column].unique()))   
    sample_list = [] 
    spare_list = []
    groups = df.groupby(column)
    
    for label in df[column].unique(): 
        
        group = groups.get_group(label)
        sample_count = len(group)    
        
        if sample_count > max_size:
            strat = group[column]
            samples, spare_data = train_test_split(group, train_size = max_size, shuffle = True, random_state = 123, stratify = strat)            
            sample_list.append(samples)
            spare_list.append(spare_data)
            
        elif sample_count >= min_size:
            sample_list.append(group)
            
    df = pd.concat(sample_list, axis = 0).reset_index(drop = True)
    spare_df = pd.concat(spare_list, axis = 0).reset_index(drop = True)
    
    final_class_count = len(list(df[column].unique())) 
    
    if final_class_count != original_class_count:
        print ('El dataframe se ha reducido. Número de clases original: ', original_class_count,' Número de clases actual: ', final_class_count)
        
        
    print("Dataframe trimeado:")
    groups = df.groupby('labels')
    print('{0:^30s} {1:^13s}'.format('Clase', 'Número de imágenes'))
    
    for label in df['labels'].unique():
          group = groups.get_group(label)      
          print('{0:^30s} {1:^13s}'.format(label, str(len(group))))
    
    print("Dataframe spare:")
    groups = spare_df.groupby('labels')
    print('{0:^30s} {1:^13s}'.format('Clase', 'Número de imágenes'))
        
    for label in spare_df['labels'].unique():
      group = groups.get_group(label)      
      print('{0:^30s} {1:^13s}'.format(label, str(len(group))))

    return df, spare_df

def balance(train_df, max_samples, min_samples, working_dir, image_size, fold_var):
    
    column = 'labels'
    
    train_df = train_df.copy()       

    aug_dir = os.path.join(working_dir, ('aug_fold_' + str(fold_var)))
    
    if os.path.isdir(aug_dir):
        shutil.rmtree(aug_dir)
        
    os.mkdir(aug_dir)
    
    for label in train_df['labels'].unique():    
        dir_path = os.path.join(aug_dir, label)    
        os.mkdir(dir_path)
        
     
    total = 0
    gen = ImageDataGenerator(rotation_range = 180, width_shift_range = .2, height_shift_range = .2, zoom_range = .1, horizontal_flip=True, vertical_flip = True, brightness_range=[0.7,1.3])
    
    groups=train_df.groupby('labels') # Agrupar por cada clase
    
    for label in train_df['labels'].unique():  # Por cada clase            
        group = groups.get_group(label)  # Nos quedamos con las imágenes de esa clase
        sample_count = len(group)   # Cuántas imágenes hay en esa clase
        if sample_count < max_samples: # Si hay menos imágenes:
            
            aug_img_count = 0
            delta = max_samples-sample_count  # Aumentamos el resto que falta.
            print("Resta de imágenes: ", str(delta))
            target_dir = os.path.join(aug_dir, label)  # Definimos dónde escribir las imágenes.  
            aug_gen = gen.flow_from_dataframe(group,  x_col = 'filepaths', y_col = None, target_size = image_size,
                                            class_mode = None, shuffle = False, 
                                            save_to_dir = target_dir, batch_size = 1, save_prefix = 'aug-', color_mode = 'rgb',
                                            save_format = 'jpg')
            while aug_img_count < delta:
                images = next(aug_gen)            
                aug_img_count += len(images)
                
            total += aug_img_count
            
    print('Imágenes totales creadas debido al aumento de datos:', total)
    
    # Creamos el dataset aumentado y lo unimos al de entrenamiento original.
    if total > 0:
        aug_fpaths = []
        aug_labels = []
        classlist = os.listdir(aug_dir)
        
        for klass in classlist:
            classpath = os.path.join(aug_dir, klass)     
            flist = os.listdir(classpath)    
            
            for f in flist:        
                fpath=os.path.join(classpath,f)         
                aug_fpaths.append(fpath)
                aug_labels.append(klass)
                
        Fseries = pd.Series(aug_fpaths, name='filepaths')
        Lseries = pd.Series(aug_labels, name='labels')
        aug_df = pd.concat([Fseries, Lseries], axis = 1)
        train_df = pd.concat([train_df,aug_df], axis = 0).reset_index(drop = True)
   
    print (list(train_df['labels'].value_counts()) )
    return train_df

class ASK(keras.callbacks.Callback):
    
    def __init__ (self, model, epochs,  ask_epoch): # Inicializamos el callback
        super(ASK, self).__init__()
        self.model = model               
        self.ask_epoch = ask_epoch
        self.epochs = epochs
        self.ask = True # Al ponerse true pedimos al usuario que introduzca un número de épocas inicial
        
    def on_train_begin(self, logs = None): # Esto se ejecuta al principio del entrenamiento.
        
        if self.ask_epoch == 0: 
            print('Como mínimo se debe de entrenar 1 época, por lo que se entrenará 1.', flush=True)
            self.ask_epoch = 1
            
        if self.ask_epoch >= self.epochs: # Todavía no hay que preguntar
            self.ask=False # No se le pregunta al usuario
            
        if self.epochs == 1:
            self.ask = False # Sólo se ejecuta 1 época. 
            
        else:
            print('El entrenamiento seguirá hasta la época', ask_epoch,'. Posteriormente se le le pedirá') 
            print(' escribir H para parar o volver a insertar un número de épocas para seguir entrenando.')  
            
        self.start_time = time.time() # Medimos el tiempo desde que empezó el entrenamiento.
        
    def on_train_end(self, logs = None):   # Se ejecuta al final del entrenamiento  
        tr_duration = time.time() - self.start_time   # Mide el tiempo usado en el entrenamiento    
        hours = tr_duration // 3600
        minutes = (tr_duration - (hours * 3600)) // 60
        seconds = tr_duration - ((hours * 3600) + (minutes * 60))
        msg = f'Tiempo total de entrenamiento {str(hours)} horas, {minutes:4.1f} minutos, {seconds:4.2f} segundos)'
        print (msg, flush=True) # Imprimimos el tiempo total de entrenamiento.
        
    def on_epoch_end(self, epoch, logs = None):  # Esta función se ejecuta AL FINAL DE CADA ÉPOCA
        
        if self.ask: 
            if epoch + 1 == self.ask_epoch: 
                
                print('\n Introduce H para parar el entrenamiento o un número de épocas con el que continuar entrenando.')
                
                ans = input()
                
                if ans == 'H' or ans =='h' or ans == '0': # Preguntamos condición de parada
                    print ('Has introducido ', ans, '; el entrenamiento parará en la época ', epoch+1, flush=True)
                    self.model.stop_training = True # Paramos el entrenamiento
                    
                else: # user wants to continue training
                    self.ask_epoch += int(ans)
                    if self.ask_epoch > self.epochs:
                        print('\nYou specified maximum epochs of as ', self.epochs, ' cannot train for ', self.ask_epoch, flush =True)
                    else:
                        print ('has introducido ', ans, ' épocas más. \n El entrenamiento continuará hasta la época ', self.ask_epoch, flush=True)
                        
def get_model_name(k):
    return 'fold'+str(k)+'.h5'

# Configuración de los parámetros de creación del dataset
Tamaño de imagen y número máximo y mínimo de imágenes por clase y ruta del directorio donde guardar los modelos y donde cargar los datos.

In [ ]:
# Ejecución.
max_samples = 400
min_samples = 10

sdir='/content/drive/MyDrive/data_big/'
dir = 'big'

save_dir = '/content/drive/MyDrive/saved_models/'
fold_var = 0

img_size = (224,224) 

save_dir = '/content/drive/MyDrive/saved_models/' # Dónde guardar los datos aumentados.

In [ ]:
df = get_dataframe(sdir, dir) # Cargamos el dataframe
trim_df, spare_df = trim(df, max_samples, min_samples) # Recortamos a max_samples y min_samples

train_df, test_df = train_test_split(trim_df, train_size = .8, shuffle = True, random_state = 123, stratify = trim_df['labels']) # División en entrenamiento y test.

print('Tamaño del conjunto de ENTRENAMIENTO: ', len(train_df), '  \nTamaño del conjunto de TEST: ', len(test_df))

print("Dataframe train:")
classes = list(train_df['labels'].unique())
class_count = len(classes)
groups = train_df.groupby('labels')

print('{0:^30s} {1:^13s}'.format('Clase', 'Número de imágenes'))
for label in train_df['labels'].unique():
      group = groups.get_group(label)      
      print('{0:^30s} {1:^13s}'.format(label, str(len(group))))

test_df = pd.concat([test_df, spare_df]) # Lo que ha sobrado de recortar el df de entrenamiento lo añadimos al test.
print("Tamaño del conjunto de TEST ACTUALIZADO: ", len(test_df))
test_df.to_pickle("/content/drive/MyDrive/folds/test")

train_df_labels = train_df['labels']
train_df_samples = train_df['filepaths']
print('Tamaño del conjunto de ENTRENAMIENTO: ', len(train_df), '  \nTamaño del conjunto de TEST: ', len(test_df))

# 3 fold Cross-Validation
Si se desea cambiar el número de folds, alterar n_splits.

In [ ]:
stratified_kf = StratifiedKFold(n_splits = 3, shuffle = True, random_state = 123)

train_df_fold = []
valid_df_fold = []

for train_index, val_index in stratified_kf.split(train_df_samples, train_df_labels):
    
    print(train_index.size, val_index.size)
    
    print("Número de fold: ", fold_var)

    training_data = train_df.loc[train_df.index[train_index]]
        
    working_dir = r'/content/drive/MyDrive/' 
    training_data = balance(training_data, max_samples, min_samples, working_dir, img_size, fold_var)
    #training_data.to_pickle("/content/drive/MyDrive/folds/train_df_fold" + str(fold_var) + ".pkl")

    
    train_df_fold.append(training_data)
    
    validation_data = train_df.loc[train_df.index[val_index]]
    #validation_data.to_pickle("/content/drive/MyDrive/folds/validation_df_fold" + str(fold_var) + ".pkl")

    valid_df_fold.append(validation_data)

    fold_var += 1

# Entrenamiento de los modelos en los folds
Se entrena el modelo en cada fold y se guarda un modelo en el directorio especificado anteriormente por cada fold. Se guarda el mejor modelo obtenido en el entrenamiento, teniendo en cuenta el conjunto de validación (gracias a los callbacks, que facilitan el proceso y lo hacen sencillo) \\
**Los modelos se guardan para evaluarse posteriormente y elegir el mejor.**

In [ ]:
# Hiperparámetros del modelo.
lr = .001 
dropout = 0.0
batch_size = 32
epochs = 40
pooling = "avg"
ask_epoch = 30 # Para el callback ASK. Nos preguntará en la época 30 si queremos continuar

# Entrenamiento con las redes de la familia EfficientNet

In [ ]:
img_shape = (img_size[0], img_size[1], 3)

t_and_v_gen = ImageDataGenerator()
trgen = ImageDataGenerator()

for i in range (0, fold_var - 1):
    
    print("Número de fold procesado: ", i)
    
    training_data = train_df_fold[i]
    print("Datos de entrenamiento: ", len(training_data))
    classes = list(training_data['labels'].unique())
    class_count = len(classes)
    validation_data = valid_df_fold[i]
    print("Datos de validación: ", len(validation_data))
    
    train_gen = t_and_v_gen.flow_from_dataframe(training_data, x_col = 'filepaths', y_col = 'labels', target_size = img_size,
                                       class_mode = 'categorical', color_mode = 'rgb', shuffle = True, batch_size = batch_size)
    valid_gen = t_and_v_gen.flow_from_dataframe(validation_data, x_col = 'filepaths', y_col = 'labels', target_size = img_size,
                                       class_mode = 'categorical', color_mode = 'rgb', shuffle = False, batch_size = batch_size)


    model_name = 'EfficientNetV2B0' # Para guardar el modelo con este nombre. El modelo se cambia en la línea de abajo.
    base_model=tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False, weights="imagenet",input_shape=img_shape, pooling=pooling) # Modelo a usar

    base_model.trainable = True

    x = base_model.output

    output = Dense(class_count, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    model.compile(Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy']) 
    
    model_name = save_dir+model_name+"_fold_" + str(i) + "_lr_" + str(lr) + "_bs_" + str(batch_size) + "_pooling_" + pooling + "_samples_" + str(max_samples) + ".h5"

    checkpoint = tf.keras.callbacks.ModelCheckpoint(model_name, 
							monitor='val_accuracy', verbose=1, 
							save_best_only=True, mode='max')
    rlronp = tf.keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = 0.5, patience = 2, verbose = 1)
    estop = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 6, verbose = 1, restore_best_weights = True)
    ask = ASK(model, epochs,  ask_epoch)

    callbacks = [rlronp, estop, checkpoint, ask]
    
    history = model.fit(x = train_gen,  epochs = epochs, verbose = 1, callbacks = callbacks,  validation_data = valid_gen)

# Entrenamiento con AlexNet

In [ ]:
# Hiperparámetros de AlexNet
lr = .001 
dropout = 0.0
batch_size = 32
epochs = 40
pooling = "avg"

ask_epoch = 40
img_shape = (img_size[0], img_size[1], 3)

In [ ]:
t_and_v_gen = ImageDataGenerator()
trgen = ImageDataGenerator()

for i in range (0, fold_var - 1):
    
    print("Número de fold procesado: ", i)
    
    training_data = train_df_fold[i]
    print("Datos de entrenamiento: ", len(training_data))
    classes = list(training_data['labels'].unique())
    class_count = len(classes)
    validation_data = valid_df_fold[i]
    print("Datos de validación: ", len(validation_data))
    
    train_gen = t_and_v_gen.flow_from_dataframe(training_data, x_col = 'filepaths', y_col = 'labels', target_size = img_size,
                                       class_mode = 'categorical', color_mode = 'rgb', shuffle = True, batch_size = batch_size)
    valid_gen = t_and_v_gen.flow_from_dataframe(validation_data, x_col = 'filepaths', y_col = 'labels', target_size = img_size,
                                       class_mode = 'categorical', color_mode = 'rgb', shuffle = False, batch_size = batch_size)


    model_name = 'AlexNet_borrar'
    model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(img_shape)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(class_count, activation='softmax')
    ])
    model.compile(Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy']) 
    
    model_name = save_dir+model_name+"_fold_" + str(i) + "_lr_" + str(lr) + "_bs_" + str(batch_size) + "_pooling_" + pooling + "_samples_" + str(max_samples) + ".h5"

    checkpoint = tf.keras.callbacks.ModelCheckpoint(model_name, 
							monitor='val_accuracy', verbose=1, 
							save_best_only=True, mode='max')
    rlronp = tf.keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = 0.5, patience = 2, verbose = 1)
    estop = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 6, verbose = 1, restore_best_weights = True)
    ask = ASK(model, epochs,  ask_epoch)

    callbacks = [rlronp, estop, checkpoint, ask]
    
    history = model.fit(x = train_gen,  epochs = epochs, verbose = 1, callbacks = callbacks,  validation_data = valid_gen)

# Comparativa de modelos.
Se usan todos los modelos de saved_models (o del path models en sí) y se evalúan en entrenamiento y validación. Se guarda un archivo excel los resultados obtenidos tanto en train como en validación para cada modelo. \\
Se ha utilizado este código para generar las tablas de la memoria.

In [ ]:
# comparativa de modelos
models = os.listdir('/content/drive/MyDrive/saved_models/')    

t_and_v_gen = ImageDataGenerator()

rotate_valid_datagen =  ImageDataGenerator(rotation_range = 180)

COLUMN_NAMES = ['Model', 'Fold', 'Train_accuracy', 'Train_loss', 'Valid_accuracy', 'Valid_loss', 'Valid_rotate_acc', 'Valid_rotate_loss']
results = pd.DataFrame(columns=COLUMN_NAMES)

folds =  3

for fold in range (0, folds):

  validation_data = valid_df_fold[fold]
  train_data = train_df_fold[fold]

  valid_gen = t_and_v_gen.flow_from_dataframe(validation_data, x_col = 'filepaths', y_col = 'labels', target_size=(img_size),
                                        class_mode = 'categorical', color_mode = 'rgb', batch_size = 1, shuffle = False)
  train_gen = t_and_v_gen.flow_from_dataframe(train_data, x_col = 'filepaths', y_col = 'labels', target_size=(img_size),
                                        class_mode = 'categorical', color_mode = 'rgb', shuffle = False)
  valid_r_gen = rotate_valid_datagen.flow_from_dataframe(validation_data, x_col = 'filepaths', y_col = 'labels', target_size=(img_size),
                                        class_mode = 'categorical', color_mode = 'rgb', batch_size = 1, shuffle = False)

  for model in models:
    if not model.startswith('.') and (model.find('fold_'+str(fold)) != -1):
      print("Evaluando modelo", model, " en el fold ", fold)
      path = os.path.join('/content/drive/MyDrive/saved_models/', model) 
      model_pred = tf.keras.models.load_model(path)

      train_loss, train_acc = model_pred.evaluate(train_gen, verbose=1)
      print('Accuracy en train: {:5.3f}%'.format(100 * train_acc))

      valid_loss, valid_acc = model_pred.evaluate(valid_gen, verbose=1)
      print('Accuracy en validación: {:5.3f}%'.format(100 * valid_acc))

      valid_rotate_loss, valid_rotate_acc = model_pred.evaluate(valid_r_gen, verbose=1)
      print('Accuracy en validación rotada aleatoriamente: {:5.3f}%'.format(100 * valid_rotate_acc))

      row = {'Model':model, 'Fold':fold, 'Train_accuracy':train_acc, 'Train_loss':train_loss, 'Valid_accuracy':valid_acc, 'Valid_loss':valid_loss, 'Valid_rotate_acc':valid_rotate_acc, 'Valid_rotate_loss':valid_rotate_loss}
      results = results.append(row, ignore_index=True)


# Exportación a un archivo excel de los resultados

Facilita la creación de la memoria.

In [ ]:
results_sort = results.sort_values('Model') # Ordenamos por nombre para tener los modelos juntos (fold 0, 1, 2) en el archivo Excel.
print(results_sort)
results_sort.to_excel("/content/drive/MyDrive/model_comparison.xlsx")  